# Principles of Concurrent and Distributed Programming

In [1]:
# 工作目录
%cd /mnt/d/GoogleDrive/wiki/jupyter-notebooks/Modeling/SPIN/PCDP

/mnt/d/GoogleDrive/wiki/jupyter-notebooks/Modeling/SPIN/PCDP


In [2]:
# 清理
!rm -f pan pan.*

# 1. What is Concurrent Programming?

# 2. The Concurrent Programming Abstraction

- 并发执行: 原子语句的交错(interleaving)
- 任意的交错
- 原子语句
- 正确性(correctness)
- 公平性(fairness)
- 机器码指令: 寄存器机器, 栈机器
- 易失性(volatile)变量, 非原子变量


A **concurrent program** consists of a finite set of
(sequential) processes. The processes are
written using a finite set of **atomic statements**.
The execution of a concurrent program
proceeds by executing a sequence of the
atomic statements obtained by **arbitrarily
interleaving** the atomic statements from the
processes. A **computation** is an execution
sequence that can occur as a result of the
interleaving. Computations are also called
**scenarios**.

During a computation the **control pointer** of a
process indicates the next statement that can
be executed by that process. Each process
has its own control pointer.

The **state** of a (concurrent) algorithm is a
tuple consisting of one element for each
process that is a label from that process, and
one element for each global or local variable
that is a value whose type is the same as the
type of the variable.

Let s1 and s2 be states. There is a **transition**
between s1 and s2 if executing a statement in
state s1 changes the state to s2. The statement
executed must be one of those pointed to by a
control pointer in s1.

A **state diagram** is a graph defined inductively.
The initial state diagram contains a single node
labeled with the initial state. If state s1 labels a
node in the state diagram, and if there is a
transition from s1 to s2, then there is a node
labeled s2 in the state diagram and a directed
edge from s1 to s2.
For each state, there is only one node labeled
with that state.
The set of **reachable states** is the set of states
in a state diagram.


**Safety properties** The property must always be
true.
**Liveness properties** The property must eventually
become true.


linear temporal logic (LTL)
Branching temporal logic: CTL is a branching temporal logic 

LTL is used in this book


A scenario is (**weakly**) **fair** if at any state in the
scenario, a statement that is continually
enabled eventually appears in the scenario.

工具, 语言
- BACI并发模拟器: Ben-Ari Concurrency Interpreter
- Ada
- Java
- PROMELA

Algorithm 2.1. Trivial concurrent program

convention: 
Each labeled line represents an atomic
statement.

Algorithm 2.2. Trivial sequential program
Algorithm 2.3. Atomic assignment statements
Algorithm 2.4. Assignment statements with one global reference

assumption:
Assignment statements are atomic
statements, as are evaluations of boolean
conditions in control statements.

Algorithm 2.5. Stop the loop

Algorithm 2.6. Assignment statement for a register machine
Algorithm 2.7. Assignment statement for a stack machine

An occurrence of a variable v is defined to be
**critical reference**: (a) if it is assigned to in one
process and has an occurrence in another
process, or (b) if it has an occurrence in an
expression in one process and is assigned to in
another.
A program satisfies the **limited-criticalreference (LCR)** restriction if each statement
contains at most one critical reference.

Algorithm 2.8. Volatile variables

Specifying a variable as
**volatile** instructs the compiler to load and store the
value of the variable at each use, rather than attempt
to optimize away these loads and stores.
Concurrency may also affect computations with
multiword variables. 

Algorithm 2.9. Concurrent counting algorithm

```
integer n <- 0
-
p                    | q
    integer temp     |     integer temp
p1: do 10 times      | q1: do 10 times
p2:   temp <- n      | q2:    temp <- n
p3:   n <- temp + 1  | q3:    n <- temp + 1
```

In [ ]:
# Listing 2.5. A Promela program for the counting algorithm
!spin l2.5-counting-algo.pml

      MSC: The value is 14
3 processes created


# 3. The Critical Section Problem

问题定义和假设:

- Each of N processes is executing in a infinite loop a sequence of
statements that can be divided into two subsequences: the **critical
section** and the non-critical section.

- The correctness specifications required of any solution are:
  - **Mutual exclusion** Statements from the critical sections of two
or more processes must not be interleaved.
  - **Freedom from deadlock** If some processes are trying to enter
their critical sections, then one of them must eventually
succeed.
  - **Freedom from (individual) starvation** If any process tries to
enter its critical section, then that process must eventually
succeed.

- A synchronization mechanism must be provided to ensure that the
correctness requirements are met. The synchronization mechanism
consists of additional statements that are placed before and after
the critical section. The statements placed before the critical section
are called the **preprotocol** and those after it are called the
**postprotocol**. 

- The protocols may require local or global variables, but we assume
that no variables used in the critical and non-critical sections are
used in the protocols, and vice versa.
- The critical section must **progress**, that is, once a process starts to
execute the statements of its critical section, it must eventually
finish executing those statements.
关键区域中必须有进展.
- The non-critical section need not progress, that is, if the control
pointer of a process is at or in its non-critical section, the process
may terminate or enter an infinite loop and not leave the noncritical section.
非关键区域中可以没有进展.

Algorithm 3.1. Critical section problem

Algorithm 3.2. First attempt

```
integer turn <- 1

p: 
loop forever
p1: non-critical section
p2: await turn = 1           // 等待: 我的机会
p3: critical section
p4: turn <- 2               // 他的机会

q:
loop forever
q1: non-critical section
q2: await turn = 2
q3: critical section
q4: turn <- 1
```

`turn`作为进入关键区域的权限: 值表示哪个进程持有资源.


Algorithm 3.3. History in a sequential algorithm
Algorithm 3.4. History in a concurrent algorithm
Algorithm 3.5. First attempt (abbreviated)

使用状态图证明正确性
- 互斥: YES
- 无死锁: YES
- 无饥饿: NO
  - 非关键区域可以无进展

Algorithm 3.6. Second attempt

```
boolean wantp <- false, wantq <- false
-
p:
loop forever
p1: non-critical section
p2: await wantq = false     // 等待: 他不要
p3: wantp <- true           // 我要
p4: critical section
p5: wantp <- false          // 我不要

q:
loop forever
q1: non-critical section
q2: await wantp = false
q3: wantq <- true
q4: critical section
q5: wantq <- false
```

Algorithm 3.7. Second attempt (abbreviated)

正确性:
- 互斥: NO

Algorithm 3.8. Third attempt

```
boolean wantp <- false, wantq <- false
-
p:
loop forever
p1: non-critical section
p2: wantp <- true         // 我要
p3: await wantq = false   // 等待: 他不要
p4: critical section
p5: wantp <- false        // 我不要

q:
loop forever
q1: non-critical section
q2: wantq <- true
q3: await wantp = false
q4: critical section
q5: wantq <- false
```

正确性:
- 互斥: YES
- 无死锁: NO
  - 不可出节点: 活锁

Algorithm 3.9. Fourth attempt

```
boolean wantp <- false, wantq <- false
-
p:
loop forever
p1: non-critical section
p2: wantp <- true          // 我要
p3: while wantq            // 他要时
p4:   wantp <- false       //   我不要
p5:   wantp <- true        //   我要
p6: critical section
p7: wantp <- false         // 我不要

q:
loop forever
q1: non-critical section
q2: wantq true
q3: while wantp
q4:   wantq <- false
q5:   wantq <- true
q6: critical section
q7: wantq <- false
```

正确性:
- 互斥: YES
- 无死锁: YES
- 无饥饿: NO
  - 存在环

Algorithm 3.10. Dekker's algorithm

```
boolean wantp <- false, wantq <- false
integer turn <- 1
-
p:
loop forever
p1: non-critical section
p2: wantp <- true          // 我要
p3: while wantq            // 他要时
p4:   if turn = 2          //   他的机会
p5:     wantp <- false     //     我不要
p6:     await turn = 1     //     等待: 我的机会
p7:     wantp <- true      //     我要
p8: critical section
p9: turn <- 2             // 机会给他
p10: wantp <- false       // 我不要

q:
loop forever
q1: non-critical section
q2: wantq <- true
q3: while wantp
q4:   if turn = 1
q5:     wantq <- false
q6:     await turn = 2
q7:     wantq <- true
q8: critical section
q9: turn <- 1
q10: wantq false
```

# 4. Verification of Concurrent Programs

- the specification of correctness properties: 正确性属性的描述
- inductive proof: 归纳证明
- invariant: 不变量
- a system of temporal logic: 时态逻辑的系统
  - deductive: 推论的, 演绎的
    - 在所有计算的状态上归纳
    - 只需要考虑能修改公式中原子命题真值的语句
  - specify correctness properties for model checkers
  - tools: SPIN, STeP, TLA

示例: the third attemp satisfies the mutual exclusion property

A Deductive Proof of Dekker's Algorithm
- 进展: 如果计算在一个状态满足A, 则计算必须进展到另一个状态满足B.
- 假设: 所有计算是weakly fair的.
- 赋值语句: 持有进展.
- 关键区域: 必须持有进展.
- 非关键区域: 不需要持有进展.
- 控制语句: 
  - 证明选择了某一个分支: `p4 /\ [](turn = 2) -> <> p5`

进展的证明规则:
- (a) []A -> <>B
- (b) <>[]A
结论: `<>B`

证明: 无饥饿
- 假设: `q`总是尝试进入关键区域 `[]<> ~ q1`
- `p2 -> <>P8`


the Promela Modeling Language

In [ ]:
# Listing 4.1. Dekker's algorithm in Promela
# 以Safety模式运行验证
# !spin -run -safety l4.1-dekker-algo-safety.pml



pan.c: In function ‘make_trail’:
pan.c:1942:33: warning: ‘%d’ directive writing between 1 and 10 bytes into a region of size between 1 and 512 []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-overflow=-Wformat-overflow=]8;;]
 1942 |                 sprintf(fnm, "%s%d.%s",
      |                                 ^~
pan.c:1942:30: note: directive argument in the range [1, 2147483647]
 1942 |                 sprintf(fnm, "%s%d.%s",
      |                              ^~~~~~~~~
In file included from /usr/include/stdio.h:894,
                 from pan.c:7:
/usr/include/x86_64-linux-gnu/bits/stdio2.h:38:10: note: ‘__builtin___sprintf_chk’ output 3 or more bytes (assuming 523) into a destination of size 512
   38 |   return __builtin___sprintf_chk (__s, __USE_FORTIFY_LEVEL - 1,
      |          ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
   39 |                                   __glibc_objsize (__s), __fmt,
      |                                   ~

TODO(zhoujiagen) restart after reading [The SPIN Model Checker](./SPIN-The%20SPIN%20Model%20Checker.ipynb) to know more about SPIN and PROMELA.

In [ ]:
# 以Acceptance模式运行验证, 使用时态逻辑公式
# NOT WORK
# !spin -f "[]mutex" -run -a  l4.1-dekker-algo-acceptance.pml

!spin -a l4.1-dekker-algo-acceptance.pml
!./pan -a

(never claims generated from LTL formulae are stutter-invariant)
pan:1: acceptance cycle (at depth 0)
pan: wrote l4.1-dekker-algo-acceptance.pml.trail

(Spin Version 6.5.1 -- 20 December 2019)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	+ (never_0)
	assertion violations	+ (if within scope of claim)
	acceptance   cycles 	+ (fairness disabled)
	invalid end states	- (disabled by never claim)

State-vector 36 byte, depth reached 11, errors: 1
        6 states, stored
        0 states, matched
        6 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.000	equivalent memory usage for states (stored*(State-vector + overhead))
    0.288	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage



pan: elapsed time 0 seconds


# 5. Advanced Algorithms for the Critical Section Problem

# 6. Semaphores

# 7. Monitors

# 8. Channels

# 9. Spaces

# 10. Distributed Algorithms

# 11. Global Properties

# 12. Consensus

# 13. Real-Time Systems

# A. The Pseudocode Notation

# B. Review of Mathematical Logic

# C. Concurrent Programming Problems

# D. Software Tools

- BACI, jBACI
- Spin, jSpin
- DAJ: https://github.com/motib/daj


Spin, jSpin:
- 验证过程: 生成验证器, 执行验证器, 无错误或有错误(生成踪迹文件)
  - 是否将LTL公式翻译为never claim
```
-f "..formula.."  translate LTL into never claim
```
- 三种模式
  - safety: 用于验证安全性属性(例如无死锁), 断言和不变量(例如使用`[]`公式表达的)
  - acceptance: 用于验证活跃性属性(例如使用`<>`公式表达式的无饥饿)
  - non-progress: 另一种验证活跃性属性的方法, 不显式编写LTL公式.
```
-run
		    -safety	compile for safety properties only
		    -a	    	search for acceptance cycles
		    -l	    	search for non-progress cycles
```
- 可以选择验证时是否假设weak fairness.

jSpin

```
# gcc on widows: NOT WORK
conda search gcc -c conda-forge
# use MSYS2 instead

# in Git Bash
$ pwd
/d/workspace/rtfsc/jspin
$ which gcc
/d/msys64/ucrt64/bin/gcc
$ java -jar jspin.jar
Read configuration file from current directory
```